Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import latexify

Square Lattice

The following class provides supports for simulating Anderson localization and the Hofstader model on a finite square lattice with periodic boundary conditions. The possible outputs are a plot of the eigenvalue spectrum, an eigenvector from the middle of the spectrum, the participation ratio, and the Hofstader butterfly. 

An interesting result to note is that the presence of disorder (it doesn't matter what the physical cause of it may be) kills the butterfly fractal. This idea is introduced in: cite Bhatt

In [ ]:
class Square_Hamiltonian:
    """ Square lattice hamiltonian class """
    def __init__(self, length: int, t: float, disorder: float, phi: float):
        self.L = length  # Set hamiltonian dimensions
        self.matrix = np.zeros((self.L * self.L, self.L * self.L), dtype=complex)  # Initialize hamiltonian matrix
        self.disorder = disorder  # Set disorder parameter
        self.t = t # Set hopping amplitude
        self.phi = phi # Set magnetic field parameter

    def disorder_setter(self, i):
        # Incorporate the disorder parameter into matrix elements
        eps_i = self.disorder * (2 * np.random.rand() - 1)  # Random on-site energy
        self.matrix[i, i] = eps_i

    def hamiltonian_setter(self, i, j, phi=None):
        n = i * self.L + j
        # Vertical hopping
        m = ((i + 1) % self.L) * self.L + j
        self.matrix[n, m] = self.t

        # Horizontal hopping
        m = i * self.L + (j + 1) % self.L

        B = np.exp(1j * 2 * np.pi * (self.phi if phi is None else phi) * i)
        self.matrix[n, m] = B * self.t

        if self.disorder != 0:
            self.disorder_setter(n)


    def construct_matrix(self, phi=None):
        # We want to iterate through i and j
        for i, j in np.ndindex((self.L, self.L)):
            self.hamiltonian_setter(i, j, phi)

        if self.phi != 0:
            self.H = (self.matrix + self.matrix.T.conj()) / 2 # symmetrize
        else:
            self.H = self.matrix + self.matrix.T  # symmetrize
        print(self.H)
        self.evals, self.evecs = np.linalg.eigh(self.H)  # calculate eigenvalues and eigenvectors

    """ basic plotting functions """

    def plot_evals(self):
        # Plot eigenvalues of hamiltonian matrix
        plt.plot(self.evals, '.')
        plt.ylabel(r'$E_i$')
        plt.xlabel(r'$i$')
        plt.show()

    def plot_evec(self):
        # Plot an eigenvector
        self.psi = self.evecs[:,self.L//2] # Some eigenvector in the middle of the spectrum
        plt.plot(np.abs(self.psi)**2)
        plt.xlabel('x')
        plt.ylabel(r'$ |\psi(x)|^2$')
        plt.show()

    def plot_evec_disorder(self):
        # Plot an eigenvector supposing disorder exists
        self.psi = self.evecs[:,self.L//2] 
        fig, ax = plt.subplots(2,1,sharex=True)
        ax[0].plot(np.abs(self.psi)**2)
        ax[1].semilogy(np.abs(self.psi)**2)
        ax[1].set_xlabel('x')
        ax[0].set_ylabel(r'$ |\psi(x)|^2$')
        ax[1].set_ylabel(r'$ |\psi(x)|^2$')
        plt.show()

    def plot_pr(self):
        self.PR = 1./np.sum(np.abs(self.evecs)**4, axis=0) # 'evecs' is a matrix of $\psi_i(x)$ amplitudes, 1st axis is x. This does the sum over x.
        plt.plot(self.evals, self.PR, 'o')
        plt.xlabel('E')
        plt.ylabel('PR(E)')
        plt.show()

    """ hofstader butterfly support """

    # A gcd function to set irrational values for phi
    def gcd(self, a, b):
        if b == 0:
            return a
        return self.gcd(b, a % b)
    # Plot the Hofstadter butterfly

    def plot_butterfly(self):
        plt.figure(figsize=(8, 6))
        for p in range(1, self.L + 1):
          for q in range(1, self.L + 1):
              if q > p and self.gcd(p, q) == 1:
                  nphi = p / q
                  self.construct_matrix(phi = nphi) # Rebuild hamiltonian for each (p,q)
                  # Plot each energy level for the given phi
                  plt.plot(self.evals, np.full_like(self.evals, nphi), 'o', c="black", markersize=0.1)

        # Plot title and labels
        plt.xlabel(r'$\epsilon$', fontsize=15)
        plt.ylabel(r'$\phi$', fontsize=15)
        plt.title(r'Hofstadter Butterfly for $L=' + str(self.L) + '$', fontsize=15)
        plt.grid()
        plt.show()


Honeycomb Lattice

In [ ]:
L = 3 # lattice sites
t = 1 # hopping amplitude
# keeping to periodic boundary conditions
# the AA and BB blocks should be zeros for now since I'm not playing around with different on-site potentials
AA = np.zeros((L, L), dtype= complex) # AA block
BB = np.zeros((L, L), dtype= complex) # BB block

# now I want to populate the matrixes corresponding to the AB and BA blocks using seperate functions

def AB_block(L, t):
  AB = np.zeros((L, L), dtype= complex) # initialize AB
  for i, j in np.ndindex((L, L)):
  # Vertical hopping: Connect (i, j) to (i + 1, j) if (i + j) % 2 == 0 (ie if the sum of indices is even since the remainder is zero)
    if (i + j) % 2 == 0:
      m = (i + 1) % L
      AB[i, j] = t

    # Horizontal hopping: Connect (i, j) to (i, j + 1)
    m = (j + 1) % L
    AB[i, m] = t

  return AB

def BA_block(L,t):
  return AB_block(L, t).T.conjugate()

AB = AB_block(L,t)
BA = BA_block(L,t)
# construct complete hamiltonian using the four blocks

upper = np.concatenate((AA, AB), axis=1)
lower = np.concatenate((BA, BB), axis=1)
H = np.concatenate((upper, lower), axis=0)
print(H) # just to check the hamiltonian

In [ ]:
# need to make an OOP version and add butterfly supports 

Triangular Lattice

Kagome Lattice

Oblique Lattice 